In [1]:
import pandas as pd
import glob
import os
import numpy as np
from functools import reduce
from datetime import datetime

def fractional_year_to_date(fractional_year):
    """Convert fractional year to datetime object."""
    year = int(fractional_year)
    month = int(round((fractional_year - year) * 12)) + 1    
    if month > 12:
        month -= 12
        year += 1
    return datetime(year, month, 1)

def load_and_process_historical_data():
    """Load and process historical data from Excel."""
    df = pd.read_excel("historical_data/historical_special_coverage_exp_db.xlsx", index_col=[0])
    df = df.query("index_type == 'Endeks' and (code == '2. Mallar' | code == '3. Hizmet')")
    df = df[~df.code.str.contains("Goods|Services")]
    df = df[~df['type_eng'].str.contains("tobacco", case=False, na=False)]
    
    keywords = ['energy', 'telecommunication', "bread", "other processed", "durable", "core"]
    pattern = '|'.join(keywords)
    df = df[df['type_eng'].str.contains(pattern, case=False, na=False)]
    
    df = df[["data_date", "type_eng", "value"]].set_index("data_date")
    df = df.pivot(columns='type_eng', values='value')
    df = df[df.index >= '2019-12-01']
    
    return df

def load_ma_results(folder_path):
    """Load and merge MA results from Excel files."""
    excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
    dfs = []
    
    for file in excel_files:
        df_ = pd.read_excel(file)
        df_ = df_[['Date', 'Seasonally_Adjusted']]
        base_name = os.path.splitext(os.path.basename(file))[0]
        df_.rename(columns={'Seasonally_Adjusted': f'{base_name}'}, inplace=True)
        dfs.append(df_)
    
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), dfs)
    merged_df = merged_df.sort_values('Date').reset_index(drop=True)
    merged_df['Date'] = merged_df['Date'].apply(fractional_year_to_date)
    merged_df['Date'] = pd.to_datetime(merged_df['Date'])
    merged_df['Date'] = merged_df['Date'].dt.to_period('M').dt.to_timestamp()
    merged_df.rename(columns={"Date": "data_date"}, inplace=True)
    merged_df.set_index("data_date", inplace=True)
    
    return merged_df

def load_alcohol_data():
    """Load and process alcohol data."""
    df_alkol = pd.read_excel("historical_data/historical_cpi_exp_db.xlsx", index_col=[0])
    df_alkol = df_alkol[(df_alkol.cpi_index == 2) | (df_alkol.cpi_index == 12310)]
    df_alkol = df_alkol[["item", "value"]]
    df_alkol = df_alkol.pivot(columns='item', values='value')
    
    return df_alkol

def create_monthly_index(df):
    """Create monthly index with December as base (100)."""
    monthly_index_df = pd.DataFrame(index=df.index)
    
    for col in df.columns:
        base_series = df[col]
        index_series = pd.Series(index=base_series.index, dtype='float64')
        years = sorted(base_series.index.year.unique())
        
        for i, year in enumerate(years):
            year_data = base_series[base_series.index.year == year].sort_index()
            
            if 12 not in year_data.index.month:
                continue
            
            dec_date = year_data[year_data.index.month == 12].index[0]
            dec_val = base_series[dec_date]
            index_series[dec_date] = 100
            
            if i + 1 < len(years):
                next_year = years[i + 1]
                next_year_data = base_series[base_series.index.year == next_year].sort_index()
                prev_val, prev_idx_val = dec_val, 100
                
                for date in next_year_data.index:
                    curr_val = base_series[date]
                    curr_idx_val = prev_idx_val * (curr_val / prev_val)
                    index_series[date] = curr_idx_val
                    prev_val, prev_idx_val = curr_val, curr_idx_val
        
        monthly_index_df[col] = index_series
    
    return monthly_index_df.astype(float)

def calculate_weighted_index(monthly_index_df, weights_path):
    """Calculate weighted general index after dropping certain components."""
    
    # Load weights and convert to fractions
    weights_df = pd.read_excel(weights_path, index_col="Tarih", parse_dates=["Tarih"])
    weights = weights_df / 100.0
    weights = weights.div(weights.sum(axis=1), axis=0)
    # Align both datasets on index (dates) and columns (CPI components)
    df_aligned, weights_aligned = monthly_index_df.align(weights, join="inner", axis=0)
    
    # Ensure column alignment (drop any extra columns in weights)
    weights_aligned = weights_aligned[df_aligned.columns]
    
    # Columns to drop
    drop_cols = [
        "Taze Meyve ve Sebze",
        "Enerji",
        "Alkollü içecekler ve tütün",
        "Diğer işlenmemiş gıda",
        "Mücevherat"
    ]
    
    # Drop from both dataframes (if present)
    df_aligned = df_aligned.drop(columns=[col for col in drop_cols if col in df_aligned.columns], errors='ignore')
    weights_aligned = weights_aligned.drop(columns=[col for col in drop_cols if col in weights_aligned.columns], errors='ignore')
    
    # Re-normalize weights so each row sums to 1
    weights_aligned = weights_aligned.div(weights_aligned.sum(axis=1), axis=0)
    print(weights_aligned)
    # Compute general index as weighted sum
    general_index = (df_aligned * weights_aligned).sum(axis=1)
    
    # Normalize to 100 based on the first value
    base_value = general_index.iloc[0]
    general_index_normed = (general_index / base_value) * 100
    
    return general_index_normed, df_aligned, weights_aligned


def calculate_monthly_pct_change(general_index_normed, df_aligned, weights_aligned):
    """Calculate monthly percentage changes with December adjustments."""
    general_index_pct_change = general_index_normed.pct_change() * 100
    
    for year in general_index_normed.index.year.unique():
        try:
            dec = pd.Timestamp(f"{year}-12-01")
            nov = pd.Timestamp(f"{year}-11-01")
            
            if dec in general_index_normed.index and nov in general_index_normed.index:
                dec_weights = weights_aligned.loc[dec]
                dec_values = df_aligned.loc[dec]
                base_value = general_index_normed.iloc[0]
                
                dec_index = (dec_values * dec_weights).sum()
                dec_index_normed = (dec_index / base_value) * 100
                pct_change_dec = ((dec_index_normed - general_index_normed.loc[nov]) / general_index_normed.loc[nov]) * 100
                
                general_index_pct_change.loc[dec] = pct_change_dec
        except KeyError:
            continue
    
    return general_index_pct_change

def calculate_december_yoy_indices(df, weights_df):
    """Calculate year-over-year indices for December months."""
    df_december = df[df.index.month == 12]
    index_dfs = []
    
    for i in range(1, len(df_december)):
        base = df_december.iloc[i-1]
        current = df_december.iloc[i]
        index_values = (current / base) * 100
        index_values.name = df_december.index[i]
        index_dfs.append(index_values)
    
    df_index = pd.DataFrame(index_dfs)
    df_index.index.name = "data_date"
    
    weighted_index = {}
    for year in df_index.index.year.unique():
        dec_date = pd.Timestamp(year=year, month=12, day=1)
        if dec_date in weights_df.index:
            weights = weights_df.loc[dec_date]
            indices = df_index.loc[df_index.index.year == year].iloc[0]
            common_cols = weights.index.intersection(indices.index)
            
            weighted_sum = np.sum(indices[common_cols] * weights[common_cols])
            weighted_index[year] = weighted_sum
        else:
            weighted_index[year] = np.nan
    
    return pd.DataFrame.from_dict(weighted_index, orient='index', columns=['Weighted Index'])

def main():
    """Main function to execute the entire pipeline."""
    # Load and process data
    df = load_and_process_historical_data()
    ma_results = load_ma_results("ma_results")
    #alcohol_data = load_alcohol_data()
    
    # Merge data
    df = df.merge(ma_results, left_index=True, right_index=True)
    #df = df.merge(alcohol_data, left_index=True, right_index=True)
    
    # Rename columns
    column_mapping = {
        "Energy": "Enerji",
        "taze_sebze_meyve_sa_series_alternatives": "Taze Meyve ve Sebze",
        "Bread and cereals": "Ekmek ve tahıllar",
        "kira_sa_series_alternatives": "Kira",
        "giyim_ayakkabi_sa_series_alternatives": "Giyim ve ayakkabı",
        "diger_hizmetler_sa_series_alternatives": "Diğer hizmetler",
        "Telecommunication services(5)": "Haberleşme hizmetleri",
        "diger_islenmemis_sa_series_alternatives": "Diğer işlenmemiş gıda",
        "lokanta_sa_series_alternatives": "Lokanta ve oteller",
        "ulastirma_sa_series_alternatives": "Ulaştırma hizmetleri",
        "Food and non-alcoholic beverages": "Alkollü içecekler ve tütün",
        "Durable goods (excluding gold)": "Dayanıklı mallar (Altın hariç)",
        "Other processed food": "Diğer işlenmiş gıda",
        "Other core goods": "Diğer temel mallar"
    }
    df = df.rename(columns=column_mapping)
    df = df[df.columns[df.columns.str.contains("Ekmek|Diğer işlenmiş")]]
    print(df.columns)
    # Create indices
    monthly_index_df = create_monthly_index(df)
    general_index_normed, df_aligned, weights_aligned = calculate_weighted_index(
        monthly_index_df, "weights/weight_special_coverage_islenmis_gida.xlsx"
    )
    monthly_pct_change = calculate_monthly_pct_change(general_index_normed, df_aligned, weights_aligned)
    
    # Create initial result
    result = pd.DataFrame({
        "general_index": general_index_normed,
        "monthly_pct_change": monthly_pct_change
    })
    
    # Calculate December YoY indices
    weights_df = pd.read_excel("weights/weight_special_coverage_islenmis_gida.xlsx", index_col="Tarih", parse_dates=["Tarih"])
    #weights_df = weights_df.div(weights_df.sum(axis=1), axis=0)
    df_weighted = calculate_december_yoy_indices(df, weights_df)
    df_weighted.index = pd.to_datetime(df_weighted.index.astype(str) + '-12-01')
    df_weighted.index.name = 'Date'
    
    # Merge and adjust December values
    result.index = pd.to_datetime(result.index)
    merged = result.merge(df_weighted, how='left', left_index=True, right_index=True)
    merged['adjusted_monthly_pct_change'] = merged['monthly_pct_change']
    
    # Adjust December monthly changes
    december_mask = merged.index.month == 12
    for date in merged[december_mask & merged['Weighted Index'].notna()].index:
        prev_month = date - pd.DateOffset(months=1)
        if prev_month in merged.index:
            prev_index = merged.loc[prev_month, 'general_index']
            weighted = merged.loc[date, 'Weighted Index']
            pct_change = (weighted / prev_index - 1) * 100
            merged.loc[date, 'adjusted_monthly_pct_change'] = pct_change
    
    return merged


final_result = main()
final_result

Index(['Ekmek ve tahıllar', 'Diğer işlenmiş gıda'], dtype='object')
            Ekmek ve tahıllar  Diğer işlenmiş gıda
2019-12-01           0.329128             0.670872
2020-01-01           0.332564             0.667436
2020-02-01           0.332564             0.667436
2020-03-01           0.332564             0.667436
2020-04-01           0.332564             0.667436
...                       ...                  ...
2025-06-01           0.331837             0.668163
2025-07-01           0.331837             0.668163
2025-08-01           0.331837             0.668163
2025-09-01           0.331837             0.668163
2025-10-01           0.331837             0.668163

[71 rows x 2 columns]


,general_index,monthly_pct_change,Weighted Index,adjusted_monthly_pct_change
2019-12-01,100.000000,NaN,NaN,NaN
2020-01-01,101.464207,1.464207,NaN,1.464207
2020-02-01,102.353741,0.876697,NaN,0.876697
2020-03-01,103.204438,0.831134,NaN,0.831134
2020-04-01,104.658409,1.408826,NaN,1.408826
...,...,...,...,...
2025-06-01,115.396222,0.969019,NaN,0.969019
2025-07-01,117.909860,2.178267,NaN,2.178267
2025-08-01,122.428656,3.832416,NaN,3.832416
2025-09-01,127.200273,3.897468,NaN,3.897468


In [2]:
final_result.to_excel("diger_islenmis.xlsx")